In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
%python
spark.sql(f"""
    CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_capillary_membership 
    TBLPROPERTIES (
        'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
        'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
    )
    AS
    SELECT
        TO_DATE(FROM_UTC_TIMESTAMP(CAST(created_at / 1000 AS TIMESTAMP), 'Asia/Kolkata')) AS CALDAY,
        data.firstname fullname,
        CAST(data.customerIdentifiers.instore.mobile AS STRING) mobile_number,
        CAST(data.customerIdentifiers.customerId AS STRING) user_id,
        UPPER(get(FILTER(data.extendedFields, x -> x.key = 'gender'), 0).value) gender,
        CAST(get(FILTER(data.extendedFields, x -> x.key = 'dob_date'), 0).value AS DATE) dob,
        get(FILTER(data.extendedFields, x -> x.key = 'acquisition_channel'), 0).value reg_source,
        get(FILTER(data.customFields, x -> x.key = 'staff_id'), 0).value staff_id,
        get(FILTER(data.customFields, x -> x.key = 'masan_customer_id'), 0).value customer_id,
        get(FILTER(data.customFields, x -> x.key = 'masan_referral_code'), 0).value masan_referral_code,
        UPPER(CASE data.enteredBy.store.code
            WHEN 'masan.wintel.prod' THEN 'WINTEL'
            WHEN 'masan.winmart.prod' THEN 'MASANWINMARTPROD'
            WHEN 'masan.wincare.prod' THEN 'MASANWINCAREPROD'
            WHEN 'masan.phuclongecom.prod' THEN 'MASANPHUCLONGECOMPROD'
            WHEN 'masan.meatdeli.prod' THEN 'MASANMEATDELIPROD'
            ELSE element_at(SPLIT('data.enteredBy.store.code', '\\.'),3)
        END) store_id,
        data.enteredBy.store.name store_name,
        data.customerIdentifiers.customerId merchant_customer_id,
        CASE data.enteredBy.store.code
            WHEN 'masan.wintel.prod' THEN 'WINTEL'
            WHEN 'masan.winmart.prod' THEN 'MASANWINMARTPROD'
            WHEN 'masan.wincare.prod' THEN 'MASANWINCAREPROD'
            WHEN 'masan.phuclongecom.prod' THEN 'MASANPHUCLONGECOMPROD'
            WHEN 'masan.meatdeli.prod' THEN 'MASANMEATDELIPROD'
            ELSE element_at(SPLIT('data.enteredBy.store.code', '\\.'),3)
        END merchant_store_id,
        data.enteredBy.store.name merchant_store_name,
        DATE(CURRENT_TIMESTAMP() + interval 7 hours) as proc_date
    FROM {catalog_name}.udp_wcm_silver_realtime.capillary_customer_add_event
    WHERE data.enteredBy.store.code NOT IN ('masan.wintel.dev', 'masan.ecommerce')
""")